# ATTN: This script should be run AFTER spacy sentence splitting is completed.


# Explode DF on sentences


### START HERE IF SOURCING FROM DF_JOBS_SENTENCIZED
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [ ]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [ ]:
from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8


In [ ]:
df_jobs = pd.read_pickle(f'{df_save_dir}df_jobs_sentencized.pkl')


In [ ]:
# len=17599
df_jobs.info()

In [ ]:
# Job Ad info
get_df_info(df_jobs, ivs_all=ivs_all)


In [ ]:
df_jobs['Job Description spacy_sentencized'].head()


In [ ]:
df_jobs['Job Description spacy_sentencized'].describe()


In [ ]:
# Explode df so that every row is one sentence
df_jobs = df_jobs.explode('Job Description spacy_sentencized', ignore_index=True)


In [ ]:
# len = 408599
len(df_jobs)


In [ ]:
df_jobs.info()


In [ ]:
df_jobs['Job Description spacy_sentencized'].head()

In [ ]:
df_jobs['Job Description spacy_sentencized'].tail()

In [ ]:
df_jobs['Job Description spacy_sentencized'].describe()


In [ ]:
# Get longest sentence, sentence does not have any punctuation
df_jobs['Job Description spacy_sentencized'].loc[df_jobs['Job Description spacy_sentencized'].apply(len).idxmax()]


In [ ]:
# Get length of longest sentence, len=2497
df_jobs['Job Description spacy_sentencized'].apply(len).max()


In [ ]:
# Get length of shortest sentence, len=4
df_jobs['Job Description spacy_sentencized'].apply(len).min()

In [ ]:
# Get shortest sentence, its a punctuation (fullstop)
df_jobs['Job Description spacy_sentencized'].loc[df_jobs['Job Description spacy_sentencized'].apply(len).idxmin()]


In [ ]:
df_jobs['Job Description spacy_sentencized'] = df_jobs['Job Description spacy_sentencized'].apply(lambda x: unicodedata.normalize('NFKD', x.encode('ascii', 'ignore').decode('utf-8', 'ignore')))


In [ ]:
assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_jobs_sentencized_exploded.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_sentencized_exploded.csv', index=False)

# Validate sentencization and get word frequencies


### START HERE IF SOURCING FROM DF_JOBS_SENTENCIZED_EXPLODED
### PLEASE SET CORRECT DIRECTORY PATHS BELOW

In [ ]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [ ]:
from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8


In [ ]:
def get_word_num_and_frequency(row, text_col):

    with open(f'{data_dir}punctuations.txt', 'rb') as f:
        custom_punct_chars = pickle.load(f)
    row['Job Description num_words'] = len(str(row[text_col]).split())
    row['Job Description num_unique_words'] = len(set(str(row[text_col]).split()))
    row['Job Description num_chars'] = len(str(row[text_col]))
    row['Job Description num_chars_no_whitespact_and_punt'] = len(
        [
            c
            for c in str(row[text_col])
            if c not in custom_punct_chars and c not in list(string.punctuation) and c in list(string.printable) and c not in list(string.whitespace) and c != ' '
        ]
    )
    row['Job Description num_punctuations'] = len(
        [
            c
            for c in str(row[text_col])
            if c in custom_punct_chars and c in list(string.punctuation) and c in list(string.printable) and c not in list(string.whitespace) and c != ' '
        ]
    )

    return row


In [ ]:
df_jobs = pd.read_pickle(f'{df_save_dir}df_jobs_sentencized_exploded.pkl')


In [ ]:
df_jobs['Job Description spacy_sentencized'] = df_jobs['Job Description spacy_sentencized'].apply(lambda x: unicodedata.normalize('NFKD', x.encode('ascii', 'ignore').decode('utf-8', 'ignore')))


In [ ]:
df_jobs['Job Description spacy_sentencized'] = df_jobs[
    'Job Description spacy_sentencized'
].apply(lambda x: x if isinstance(x, str) else ast.literal_eval(x))


In [ ]:
df_jobs['Job Description spacy_sentencized'] = df_jobs['Job Description spacy_sentencized'].apply(lambda x: x if isinstance(x, str) else np.nan)


In [ ]:
df_jobs['Job Description spacy_sentencized'] = df_jobs['Job Description spacy_sentencized'].astype(str)


In [ ]:
#len=408599
len(df_jobs)


In [ ]:
df_jobs['Job Description spacy_sentencized'] = df_jobs[
    'Job Description spacy_sentencized'
].apply(
    lambda x: ' '.join(x.split()).strip()
    if bool(x)
    and isinstance(x, str)
    and len(x.split()) > 0
    else np.nan
)


In [ ]:
df_jobs['Job Description spacy_sentencized'] = df_jobs[
    'Job Description spacy_sentencized'
].apply(
    lambda x: ' '.join(re.split(pattern, x)).strip()
    if bool(x)
    and isinstance(x, str)
    and len(x.split()) > 0
    else np.nan
)


In [ ]:
#len=408599
len(df_jobs)

In [ ]:
df_jobs = df_jobs.dropna(subset=['Job ID', 'Job Description spacy_sentencized'])


In [ ]:
# len=408599
len(df_jobs)


In [ ]:
# Drop na, empty strings, punctuations, and keep only sentences of length more than 0
with open(f'{data_dir}punctuations.txt', 'rb') as f:
    custom_punct_chars = pickle.load(f)

df_jobs = df_jobs.dropna(subset=['Job Description'])
df_jobs = df_jobs.loc[
    (df_jobs['Job Description spacy_sentencized'].isnull() == False)
    & (df_jobs['Job Description spacy_sentencized'].isna() == False)
    & (df_jobs['Job Description spacy_sentencized'].notnull() == True)
    & (df_jobs['Job Description spacy_sentencized'].notna() == True)
    & (df_jobs['Job Description spacy_sentencized'].astype(bool) == True)
    & (df_jobs['Job Description spacy_sentencized'].apply(len) > 0)
    & (pd.isna(df_jobs['Job Description spacy_sentencized']) == False)
    & (df_jobs['Job Description spacy_sentencized'].astype(str).isin(custom_punct_chars) == False)
    & (df_jobs['Job Description spacy_sentencized'].astype(str).isin(non_whitespace_nan_list) == False)
    & (df_jobs['Job Description spacy_sentencized'].astype(str).isin(list(string.punctuation)) == False)
    & (df_jobs['Job Description spacy_sentencized'].astype(str).isin(list(string.whitespace)) == False)
]
df_jobs = df_jobs.dropna(subset=['Job Description'])


In [ ]:
# len=408599
len(df_jobs)


In [ ]:
df_jobs['Job Description spacy_sentencized'].head()


In [ ]:
df_jobs['Job Description spacy_sentencized'].describe()


In [ ]:
df_jobs['Job Description spacy_sentencized'] = df_jobs['Job Description spacy_sentencized'].dropna()


In [ ]:
# len=408599
len(df_jobs)


In [ ]:
df_jobs['Job Description spacy_sentencized'].head()


In [ ]:
# count=408599, unique=217622
df_jobs['Job Description spacy_sentencized'].describe()


In [ ]:
%%time
with open(f'{data_dir}punctuations.txt', 'rb') as f:
    custom_punct_chars = pickle.load(f)

for idx, row in tqdm.tqdm(df_jobs['Job Description spacy_sentencized'].items()):
    cleaned_job_sentence = row.split()
    cleaned_job_sentence = list(
        filter(
            lambda items: (
                item is not None
                and isinstance(item, str)
                and item not in nan_list
                and item not in custom_punct_chars
                and item not in list(string.punctuation)
                and item not in list(string.whitespace)
                and item in list(string.printable)
                for item in items
            ),
            cleaned_job_sentence
        )
    )
    cleaned_job_sentence = ' '.join(cleaned_job_sentence).strip()

    if len(cleaned_job_sentence.split()) > 2:
        df_jobs.loc[idx, 'Job Description spacy_sentencized'] = cleaned_job_sentence
    else:
        df_jobs = df_jobs.drop(idx)


In [ ]:
# len = 347914
len(df_jobs)

In [ ]:
df_jobs['Job Description spacy_sentencized'].head()


In [ ]:
# count=347914, unique=202343
df_jobs['Job Description spacy_sentencized'].describe()


In [ ]:
df_jobs['Job Description spacy_sentencized'] = df_jobs['Job Description spacy_sentencized'].dropna()


In [ ]:
df_jobs['Job Description spacy_sentencized'] = df_jobs['Job Description spacy_sentencized'].apply(lambda x: unicodedata.normalize('NFKD', x.encode('ascii', 'ignore').decode('utf-8', 'ignore')))


In [ ]:
# len = 347914
len(df_jobs)


In [ ]:
df_jobs['Job Description spacy_sentencized'].head()


In [ ]:
# count=347914, unique=202343
df_jobs['Job Description spacy_sentencized'].describe()


In [ ]:
# Remove duplicate sentences from the same job ad
df_jobs = df_jobs.drop_duplicates(subset=['Job ID', 'Job Description spacy_sentencized'])


In [ ]:
# len = 307464
len(df_jobs)


In [ ]:
# count=307464, unique=202343
df_jobs['Job Description spacy_sentencized'].describe()


In [ ]:
df_jobs['Job Description spacy_sentencized'] = df_jobs['Job Description spacy_sentencized'].dropna()


In [ ]:
# len=307464
len(df_jobs)


In [ ]:
df_jobs['Job Description spacy_sentencized'].head()


In [ ]:
# Drop na, empty strings, punctuations, and keep only sentences of length more than 0
with open(f'{data_dir}punctuations.txt', 'rb') as f:
    custom_punct_chars = pickle.load(f)

df_jobs = df_jobs.dropna(subset=['Job Description spacy_sentencized'])
df_jobs = df_jobs.loc[
    (df_jobs['Job Description spacy_sentencized'].isnull() == False)
    & (df_jobs['Job Description spacy_sentencized'].isna() == False)
    & (df_jobs['Job Description spacy_sentencized'].notnull() == True)
    & (df_jobs['Job Description spacy_sentencized'].notna() == True)
    & (df_jobs['Job Description spacy_sentencized'].astype(bool) == True)
    & (df_jobs['Job Description spacy_sentencized'].astype(str).apply(len) > 0)
    & (pd.isna(df_jobs['Job Description spacy_sentencized']) == False)
]


In [ ]:
# len=307464
len(df_jobs)

In [ ]:
# count=307464, unique=202343
df_jobs['Job Description spacy_sentencized'].describe()


In [ ]:
df_jobs['Job Description spacy_sentencized'] = df_jobs['Job Description spacy_sentencized'].apply(lambda x: ' '.join(re.split(pattern, x)))


In [ ]:
# Get the longest sentence
df_jobs['Job Description spacy_sentencized'].loc[df_jobs['Job Description spacy_sentencized'].apply(len).idxmax()]


In [ ]:
# len=2496
df_jobs['Job Description spacy_sentencized'].apply(len).max()


In [ ]:
# Get the shortest sentence
df_jobs['Job Description spacy_sentencized'].loc[df_jobs['Job Description spacy_sentencized'].apply(len).idxmin()]


In [ ]:
# len=6
df_jobs['Job Description spacy_sentencized'].apply(len).min()


In [ ]:
%%time
# Get sentence word frequencies
df_jobs = df_jobs.progress_apply(
    lambda row: get_word_num_and_frequency(
        row=row, text_col='Job Description spacy_sentencized'
    ), 
    axis='columns',
)


In [ ]:
df_jobs[
    [
        'Job Description spacy_sentencized',
        'Job Description num_words', 'Job Description num_unique_words',
        'Job Description num_chars', 'Job Description num_chars_no_whitespact_and_punt'
    ]
].head()

In [ ]:
# Get the longest sentence words = 349
df_jobs['Job Description num_words'].max()


In [ ]:
# Get the sentence with the max words
df_jobs['Job Description spacy_sentencized'].loc[df_jobs['Job Description num_words'].idxmax()]


In [ ]:
# Get the longest sentence words = 3
df_jobs['Job Description num_words'].min()


In [ ]:
# Get the sentence with the min words
df_jobs['Job Description spacy_sentencized'].loc[df_jobs['Job Description num_words'].idxmin()]


In [ ]:
df_jobs['Job Description spacy_sentencized_lower'] = df_jobs['Job Description spacy_sentencized'].progress_apply(
    lambda job_sentence: job_sentence.strip().lower()
)


In [ ]:
df_jobs[['Job Description spacy_sentencized', 'Job Description spacy_sentencized_lower']].head()


In [ ]:
%%time
# Add language requirement column
# Use regex to find language requirement
dutch_requirement_pattern = r'[Dd]utch [Pp]referred | [Dd]utch [Re]quired | [Dd]utch [Ll]anguage |[Pp]roficient in [Dd]utch |[Ss]peak [Dd]utch | [Kk]now [Dd]utch | [Ff]luent in [Dd]utch | [Dd]utch [Nn]ative | * [Dd]utch [Ll]evel'
english_requirement_pattern = r'[Ee]nglish [Pp]referred | [Ee]nglish [Re]quired | [Ee]nglish [Ll]anguage |[Pp]roficient in [Ee]nglish |[Ss]peak [Ee]nglish | [Kk]now [Ee]nglish | [Ff]luent in [Ee]nglish | [Ee]nglish [Nn]ative | * [Ee]nglish [Ll]evel'

lang_requirements = {
    'Dutch Requirement in Sentence': dutch_requirement_pattern, 'English Requirement in Sentence': english_requirement_pattern
}

for lang_req, lang_req_pattern in lang_requirements.items():

    if lang_req in df_jobs.columns:
        df_jobs = df_jobs.drop(columns=[lang_req])
    df_jobs[lang_req] = np.where(
        df_jobs['Job Description'].str.contains(lang_req_pattern),
        'Yes',
        'No',
    )
    df_jobs[lang_req] = df_jobs[lang_req].astype('category').cat.reorder_categories(['No', 'Yes'], ordered=True)
    df_jobs[lang_req] = pd.Categorical(df_jobs[lang_req], categories=['No', 'Yes'], ordered=True)


In [ ]:
# Yes = 8809 job sentences
df_jobs['Dutch Requirement in Job Ad'].value_counts()


In [ ]:
# Yes = 16008 job sentences
df_jobs['English Requirement in Job Ad'].value_counts()


In [ ]:
assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_jobs_sentencized_validated.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_sentencized_validated.csv', index=False)


# Use spacy to tokenize sentences

### START HERE IF SOURCING FROM DF_JOBS_SENTENCIZED_VALIDATED
### PLEASE SET CORRECT DIRECTORY PATHS BELOW

In [ ]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [ ]:
from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8


In [ ]:
df_jobs = pd.read_pickle(f'{df_save_dir}df_jobs_sentencized_validated.pkl')


In [ ]:
%%time
# Spacy tokenize
with open(f'{data_dir}punctuations.txt', 'rb') as f:
    custom_punct_chars = pickle.load(f)

df_jobs['Job Description spacy_tokenized'] = df_jobs['Job Description spacy_sentencized'].progress_apply(
    lambda job_sentence: [
        str(token.text.strip().lower())
        for token in nlp.tokenizer(job_sentence)
        if token.text is not None
        and len(token) != 0
        and len(token.text) != 0
        and bool(token)
        and bool(token.text)
        and token.text != '...'
        and not token.is_space
        and not token.is_punct
        and not token.is_quote
        and not token.is_bracket
        and not token.like_email
    ]
)

assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_jobs_tokenized_spacy.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_tokenized_spacy.csv', index=False)


In [ ]:
df_jobs['Job Description spacy_sentencized_cleaned'] = df_jobs['Job Description spacy_tokenized'].str.join(' ')


In [ ]:
df_jobs.info()


In [ ]:
df_jobs.columns


In [ ]:
# Job Sentence info
get_df_info(df_jobs, ivs_all=ivs_all)


In [ ]:
assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_jobs_tokenized_spacy.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_tokenized_spacy.csv', index=False)


# Use NLTK to tokenize sentences


### START HERE IF SOURCING FROM DF_JOBS_TOKENIZED_SPACY
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [ ]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [ ]:
from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8


In [ ]:
df_jobs = pd.read_pickle(f'{df_save_dir}df_jobs_tokenized_spacy.pkl')


In [ ]:
df_jobs.info()


In [ ]:
%%time
# Tokenize with NLTK
stop_words = set(stopwords.words('english'))
punctuations = list(string.punctuation)
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

df_jobs['Job Description nltk_tokenized'] = df_jobs['Job Description spacy_sentencized'].progress_apply(
    lambda job_sentence: [
        str(token.strip().lower()) 
        for token in word_tokenize(job_sentence) 
        if len(token) != 0 
        and token != '...' 
        and not token.lower() in nan_list
        and not token.lower() in custom_punct_chars
        and not token.lower() in set(stopwords.words('english')) 
        and not token.lower() in list(string.punctuation)
        and not token.lower() in list(string.whitespace)
        and (t.lower() in list(string.printable) for t in token.lower())
    ]
)

assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_jobs_tokenized_spacy_nltk.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_tokenized_spacy_nltk.csv', index=False)


In [ ]:
df_jobs['Job Description nltk_tokenized'].head()


In [ ]:
df_jobs['Job Description nltk_tokenized'].describe()


In [ ]:
# Keep the some of the [] rows because they contain 'Who are we', 'What we do', etc. and remove '1. ' and '2. ' etc. abd 'T)'
df_jobs = df_jobs.loc[~df_jobs['Job Description spacy_sentencized'].str.match('^\d\.|.\)')]


In [ ]:
df_jobs.info()


In [ ]:
assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_jobs_tokenized_spacy_nltk.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_tokenized_spacy_nltk.csv', index=False)


# Use gensim to tokenize sentences


### START HERE IF SOURCING FROM DF_JOBS_TOKENIZED_SPACY_NLTK
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [ ]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [ ]:
from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8


In [ ]:
df_jobs = pd.read_pickle(f'{df_save_dir}df_jobs_tokenized_spacy_nltk.pkl')


In [ ]:
df_jobs.info()


In [ ]:
%%time
df_jobs['Job Description gensim_tokenized'] = df_jobs['Job Description spacy_sentencized'].progress_apply(
    lambda sentence: preprocess_string(re.sub(pattern, '.', sentence.strip().lower()))
)

assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_jobs_tokenized_spacy_nltk_gensim.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_tokenized_spacy_nltk_gensim.csv', index=False)


In [ ]:
df_jobs['Job Description gensim_tokenized'].head()


In [ ]:
df_jobs.info()


In [ ]:
assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_jobs_tokenized_spacy_nltk_gensim.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_tokenized_spacy_nltk_gensim.csv', index=False)


# Use BERT to tokenize sentences


### START HERE IF SOURCING FROM DF_JOBS_TOKENIZED_SPACY_NLTK_GENSIM
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [ ]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [ ]:
from setup_module.imports import *  # type:ignore # isort:skip # fmt:skip # noqa # nopep8


In [ ]:
df_jobs = pd.read_pickle(f'{df_save_dir}df_jobs_tokenized_spacy_nltk_gensim.pkl')


In [ ]:
df_jobs.info()


In [ ]:
%%time
max_length = 512
returned_tensor = 'pt'
cpu_counts = torch.multiprocessing.cpu_count()
device = torch.device('mps') if torch.has_mps and torch.backends.mps.is_built() and torch.backends.mps.is_available() else torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device_name = str(device.type)
print(f'Using {device_name.upper()}')
bert_model_name = 'bert-base-uncased'
bert_tokenizer = BertTokenizerFast.from_pretrained(bert_model_name, strip_accents = True)
bert_model = BertForSequenceClassification.from_pretrained(bert_model_name).to(device)

df_jobs['Job Description bert_tokenized'] = df_jobs['Job Description spacy_sentencized'].progress_apply(
    lambda sentence: bert_tokenizer.tokenize(str(sentence))
)

assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_jobs_tokenized_spacy_nltk_gensim_bert.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_tokenized_spacy_nltk_gensim_bert.csv', index=False)


In [ ]:
df_jobs['Job Description bert_tokenized'].head()


In [ ]:
assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_jobs_for_classification.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_for_classification.csv', index=False)


In [ ]:
print(f'Saving df_jobs length {len(df_jobs)} to txt file.')
with open(f'{data_dir}df_jobs_len.txt', 'w') as f:
    f.write(str(len(df_jobs)))